# Place recognition pipeline using 3D Lidar scans and M2DP

### Import

In [1]:
import os
import glob
import numpy as np

from read_vel_hits import VelodyneSyncReader
from M2DP.m2dp import M2DP


#### 2. Init paths and reader

In [2]:
session_velodyne_sync_dir = (
    "/Users/felix/Projects/amr_hw3/velodyne_data/2012-01-08/velodyne_sync"
)

save_npz_path = (
    "/Users/felix/Projects/amr_hw3/velodyne_data/2012-01-08/"
    "m2dp_descriptors_2012-01-08.npz"
)

reader = VelodyneSyncReader()


### get all utime.bin files

In [3]:
bin_files = sorted(
    glob.glob(os.path.join(session_velodyne_sync_dir, "*.bin"))
)

print(f"Found {len(bin_files)} velodyne scans.")


Found 28127 velodyne scans.


### calculate descriptors

#### build M2DP instance to run algorithm

In [4]:
m2dp = M2DP()

In [5]:
utimes_list = []
desc_list = []

for idx, fp in enumerate(bin_files):
    utime = int(os.path.splitext(os.path.basename(fp))[0])

    points = reader.read(fp)        # (N, 3)
    descriptor, _ = m2dp.runM2DP(points)

    descriptor = np.asarray(descriptor, dtype=np.float32).ravel()

    utimes_list.append(utime)
    desc_list.append(descriptor)

    if idx % 200 == 0:
        print(
            f"[{idx}/{len(bin_files)}] "
            f"utime={utime}, points={points.shape[0]}, "
            f"desc_dim={descriptor.shape[0]}"
        )


time used for calculating a point cloud with M2DP: 0.070 seconds
[0/28127] utime=1326030975726043, points=39490, desc_dim=192
time used for calculating a point cloud with M2DP: 0.106 seconds
time used for calculating a point cloud with M2DP: 0.108 seconds
time used for calculating a point cloud with M2DP: 0.095 seconds
time used for calculating a point cloud with M2DP: 0.109 seconds
time used for calculating a point cloud with M2DP: 0.108 seconds
time used for calculating a point cloud with M2DP: 0.107 seconds
time used for calculating a point cloud with M2DP: 0.102 seconds
time used for calculating a point cloud with M2DP: 0.131 seconds
time used for calculating a point cloud with M2DP: 0.108 seconds
time used for calculating a point cloud with M2DP: 0.107 seconds
time used for calculating a point cloud with M2DP: 0.095 seconds
time used for calculating a point cloud with M2DP: 0.108 seconds
time used for calculating a point cloud with M2DP: 0.087 seconds
time used for calculating a p

KeyboardInterrupt: 

### build descriptor matrix and save

In [ ]:
utimes = np.asarray(utimes_list, dtype=np.int64)

D = desc_list[0].shape[0]
for i, d in enumerate(desc_list):
    if d.shape[0] != D:
        raise ValueError(
            f"Inconsistent descriptor dimension at index {i}: "
            f"expected {D}, got {d.shape[0]}"
        )

descriptors = np.vstack(desc_list).astype(np.float32)

np.savez_compressed(
    save_npz_path,
    utimes=utimes,
    descriptors=descriptors
)

print("Saved descriptors:")
print("  utimes:", utimes.shape)
print("  descriptors:", descriptors.shape)
print("  file:", save_npz_path)


#### check if it works - optional

In [ ]:
data = np.load(save_npz_path)
utimes_loaded = data["utimes"]
descriptors_loaded = data["descriptors"]

print("Loaded:")
print("  utimes:", utimes_loaded[:5])
print("  descriptors shape:", descriptors_loaded.shape)


#### 3. Run M2DP algorithm

In [ ]:
from M2DP.m2dp import M2DP

m2dp = M2DP()
descriptorVector, signatureMatrix = m2dp.runM2DP(PointCloudMatrix)

print("shape of descriptor vector: " + str(descriptorVector.shape))
print("shape of signature matrix: " + str(signatureMatrix.shape[0]) + " x " + str(signatureMatrix.shape[1]))